                                                                                                               בס״ד
#### This notebook is part of the main notebook for my semester project: Parking Project.

This notebook loads in the ["Open Parking and Camera Violations"](https://data.cityofnewyork.us/City-Government/Open-Parking-and-Camera-Violations/nc67-uf89) dataset and includes feature engineering so that it is fit to be merged with the "Parking Violations" dataset in the "Parking Project" notebook.

In [1]:
import pandas as pd

# download dataset of Open Parking and Camera Violations and read it into dataframe
status = pd.read_csv('guilty.zip', usecols=['Plate', 'Summons Number', 'Issue Date', 'Violation Status'])

In [2]:
originalStatus = status.copy() # make copy of original dataframe so that we can refer to it later if needed

status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27867325 entries, 0 to 27867324
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Plate             object 
 1   Summons Number    float64
 2   Issue Date        object 
 3   Violation Status  object 
dtypes: float64(1), object(3)
memory usage: 850.4+ MB


In [3]:
status.rename(columns={"Plate": "Plate ID"}, inplace=True) # rename col to match "Parking Violations" col

In [4]:
status = status.dropna(subset=['Issue Date']) # drop rows with null in Issue Date col
status = status[status["Issue Date"].str.contains('2021')] # only include rows with Issue Date in year 2021

In [5]:
# Display all possible entries in Violation Status 
status["Violation Status"].unique() 

array(['HEARING HELD-GUILTY REDUCTION', nan, 'HEARING HELD-GUILTY',
       'HEARING HELD-NOT GUILTY', 'ADMIN REDUCTION', 'HEARING PENDING',
       'APPEAL AFFIRMED', 'HEARING ADJOURNMENT', 'ADMIN CLAIM DENIED',
       'APPEAL REVERSED', 'APPEAL ABANDONED', 'ADMIN CLAIM GRANTED',
       'APPEAL MODIFIED', 'HEARING HELD-REINSTATEMENT', 'APPEAL REMANDED'],
      dtype=object)

In [6]:
not_guilty =      (status["Violation Status"] == "HEARING HELD-NOT GUILTY").sum() # number of people not guilty 
appeal_reversed = (status["Violation Status"] == "APPEAL REVERSED").sum() 
guilty =          (status["Violation Status"] == "HEARING HELD-GUILTY").sum()
guilty_reduced =  (status["Violation Status"] == "HEARING HELD-GUILTY REDUCTION").sum()
null =            status["Violation Status"].isnull().sum()

print("Not Guilty:", not_guilty)
print("Appeal Reversed:", appeal_reversed)
print("Guilty:", guilty)
print("Guilty Reduced:", guilty_reduced)
print("Null:", null)

Not Guilty: 210259
Appeal Reversed: 1827
Guilty: 515436
Guilty Reduced: 434087
Null: 3168283


Now we will create a dataframe with parking violations that are documented to be either guilty or not guilty. We will consider a violation status of "HEARING HELD-GUILTY" and "HEARING HELD-GUILTY REDUCTION" as guilty, while we will consider a status of "HEARING HELD-NOT GUILTY" and "APPEAL REVERSED" as innocent. 

Once we have a new, reduced dataframe, we can mark rows that are innocent with a 1 and rows that are guilty with a 0.

In [7]:
statusDF = status[status["Violation Status"].isin(["HEARING HELD-GUILTY", "HEARING HELD-GUILTY REDUCTION", 
                                        "HEARING HELD-NOT GUILTY", "APPEAL REVERSED"])]

In [8]:
# innocent = 1
statusDF.loc[(status['Violation Status'] == "HEARING HELD-NOT GUILTY") |
             (status['Violation Status'] == "APPEAL REVERSED"), 'Violation Status'] = 1 

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
# guilty = 0
statusDF.loc[(status['Violation Status'] == "HEARING HELD-GUILTY") |
             (status['Violation Status'] == "HEARING HELD-GUILTY REDUCTION"), 'Violation Status'] = 0 

In [10]:
# display modified dataframe
statusDF

,Plate ID,Summons Number,Issue Date,Violation Status
2,Z84JNV,4.724985e+09,03/08/2021,0
5,81220MH,4.724985e+09,03/08/2021,0
42,T672719C,4.724986e+09,03/08/2021,0
45,HLX4818,4.724986e+09,03/08/2021,0
48,77645,8.936654e+09,05/25/2021,1
...,...,...,...,...
27622695,89328MJ,4.016558e+09,02/25/2021,0
27641194,A81FPT,4.016572e+09,02/26/2021,0
27685735,68570MN,4.016577e+09,02/26/2021,1
27716862,60736MG,4.016572e+09,02/26/2021,0


Great! We are now ready to merge this dataframe with our "Parking Violations" dataframe!  

In [11]:
# save modified dataframe to csv file
statusDF.to_csv('violation_status.csv', index=False)